## Request Google Street View image metadata

Starting from this step, you will need to feed your GSV API key in order to make requests to GSV server. In next

In [ ]:
def GSV_meta_single(lat, lon, apiKey):
  """
  This function obtains GSV metadata at one single request point
  input: latitude and longitude of the request point, api key of your Google Cloud account
  output: dictionary with time, latitude, longitude and panoID of the GSV at the request point
  """
  # original request url for API call
  meta_url = "https://maps.googleapis.com/maps/api/streetview/metadata?size=600x400&location={},{}&key={}"
  # format the request url with feeded parameters
  request_url = meta_url.format(lat, lon, apiKey)
  try:
      # request and format the metadata
      response = requests.get(request_url).json()
      # return metadata if GSV exists
      return {'date':response['date'], 'lat':response['location']['lat'], 'lon':response['location']['lng'], 'panoID':response['pano_id']}
  except:
      # if no GSV is found or any other error happens, return None
      return None

In [ ]:
# construct a list to store all information
metas = []
# iterate over requestPoint DataFrame and request GSV metadata
# for i, row in tqdm(requestPoint.iterrows()):
for i, row in tqdm(requestPoint.iloc[:3].iterrows()):
    # obtain metadata for each point
    meta = GSV_meta_single(row['lat'], row['lon'], apiKey)
    # if GSV meta data is successfully downloaded and formatted, save the information
    if meta != None:
        metas.append(meta)
    # time.sleep(0.1)

# convert stored information to DataFrame
panoidDF = pd.DataFrame(metas, columns=['date', 'lat', 'lon', 'panoID'])

3it [00:01,  2.23it/s]


## Request Google Street View images

In [ ]:
def GSV_single(panoID, heading, apiKey, pitch=0, fov=90, width=600, height=400):
  """
  This function downloads GSV image with panoID and other customized parameters
  input:
    panoID: unique ID obtained from GSV meta data
    heading: horizontal angle the camera is pointing at
    pitch: vertical angle the camera is pointing at
    fov: zoom level of the image
  output:
    PIL Image object
  """
  # format the request url for downloading image
  request_url = "https://maps.googleapis.com/maps/api/streetview?size={}x{}&pano={}&heading={}&fov={}&pitch={}&key={}".format(width, height, panoID, heading, fov, pitch, apiKey)
  # request image and return
  try:
      response = requests.get(request_url)
      return Image.open(BytesIO(response.content))
  except:
      return None